# G-Research Crypto forecasting competition

### コンペ概要

目的：暗号通貨によるリターンの予測<br>
暗号通貨に関する時系列の価格履歴を使用して、価格が上がるか下がるか、どれだけの資産がリターンするかを予測します。<br>
本コンペでは、14種の暗号通貨についてのデータが提供されています。<br>

データの特性
- 時系列データ。
- 暗号通貨の変動は、他の暗号通貨や暗号通貨市場全体に対して影響している。
- 株価と同様に、通貨の価格はランダムウォークパターンに従っているようにも見える。

ベースとなる、時系列データ用CVのLGBMモデルを作成します。<br>
train.csvよりもさらに直近のデータが更新されているようです。[supplemental_train.csv](https://www.kaggle.com/c/g-research-crypto-forecasting/discussion/303302)を学習データに追加しました。<br>

**参考**
- [Tutorial to the G-Research Crypto Competition](https://www.kaggle.com/cstein06/tutorial-to-the-g-research-crypto-competition)
- [Data Description](https://www.kaggle.com/c/g-research-crypto-forecasting/data)
- [Detailed API Introduction](https://www.kaggle.com/sohier/detailed-api-introduction)

EDA：[【日本語】G-research_Data_Outline](https://www.kaggle.com/kitopl/g-research-data-outline)<br>
Base_Model：[【日本語】Bigginner_G-Research_Base_LGBM_Model](https://www.kaggle.com/kitopl/bigginner-g-research-base-lgbm-model)<br>

In [ ]:
import os
import random
import pandas as pd
import numpy as np
from lightgbm import LGBMRegressor
import matplotlib.pyplot as plt
import seaborn as sns

### データ読み込み

In [ ]:
data = pd.read_csv('../input/g-research-crypto-forecasting/train.csv')
data.head()

- **timestamp**：1970-01-01 00:00:00.000 UTCからの経過秒数を示す。データは分スケールのため、60の倍数になっているようです。
- **Asset_ID**：暗号通貨を識別するID(例：ビットコインの場合はAsset_ID = 1)。 Asset_IDと暗号通貨の対応関係はasset_details.csvで参照できます。
- **Count**：時間間隔（直前）の取引の総数。
- **Open**：指定時間間隔における始値(USD)。
- **High**：指定時間間隔における最高価格(USD)。
- **Low**：指定時間間隔における最低価格(USD)。
- **Close**：指定時間間隔における終値(USD)。
- **Volume**：購入または販売された通貨の数量。
- **VWAP**：指定時間間隔における通貨の重みづけ平均価格。
- **Target**：15分間の対数収益率から算出。
timestamp、Asset_IDの複合キーが、データを識別するインデックになります。

In [ ]:
add_data = pd.read_csv('../input/g-research-crypto-forecasting/supplemental_train.csv')
add_data.head()

In [ ]:
data = pd.concat([data, add_data], axis=0)
data.describe()

Asset_IDと暗号通貨の対応<br>
Weightは、目的変数となっているLogReturnを算出する際に使用します。

In [ ]:
asset_tmp = pd.read_csv('../input/g-research-crypto-forecasting/asset_details.csv')
asset = asset_tmp.set_index('Asset_ID').sort_index()
asset

### 特徴量エンジニアリング
チュートリアルで紹介されているupper_shadow、lower_shadowとhigh_div_low、open_sub_closeを算出します。

In [ ]:
# from the competition tutorial
def upper_shadow(df):
    return df['High'] - np.maximum(df['Close'], df['Open'])

def lower_shadow(df):
    return np.minimum(df['Close'], df['Open']) - df['Low']

def cal_features(df):
    df_cal = df[['Count', 'Open', 'High', 'Low', 'Close', 'Volume', 'VWAP']].copy()
    df_cal['upper_Shadow'] = upper_shadow(df_cal)
    df_cal['lower_Shadow'] = lower_shadow(df_cal)
    df_cal["high_div_low"] = df_cal["High"] / df_cal["Low"]
    df_cal["open_sub_close"] = df_cal["Open"] - df_cal["Close"]
    return df_cal

def Xy_split(df_train):    
    X = cal_features(df_train)
    y = df_train['Target']
    return X, y

### 学習
検証データに未来のデータを使用しないよう、[TimeSeriesSplit](https://scikit-learn.org/stable/modules/cross_validation.html#time-series-split)でCVを作成します。

In [ ]:
# weighted correlation
def corr(a, b, w):
    cov = lambda x, y: np.sum(w * (x - np.average(x, weights=w)) * (y - np.average(y, weights=w))) / np.sum(w)
    return cov(a, b) / np.sqrt(cov(a, a) * cov(b, b))

In [ ]:
from sklearn.model_selection import TimeSeriesSplit
n_splits = 3
tscv = TimeSeriesSplit(n_splits=n_splits)

X_train_dict = {}
y_train_dict = {}
X_valid_dict = {}
y_valid_dict = {}
model_dict = {}

for asset_id, asset_name in zip(asset.index, asset['Asset_Name']):
    print(f"Training model for {asset_name:<16} (ID={asset_id:<2})")
    df_asset = data[data["Asset_ID"] == asset_id].reset_index(drop=True)
    df_asset = df_asset.dropna(how='any')
    
    X_trains = []
    y_trains = []
    X_valids = []
    y_valids = []
    scores = []
    models = []
    importance = np.array([])
    
    for i, (train, valid) in enumerate(tscv.split(df_asset)):
        X_train, y_train = Xy_split(df_asset.iloc[train,:])
        X_valid, y_valid = Xy_split(df_asset.iloc[valid,:])

        model = LGBMRegressor(
            n_estimators = 1000, num_leaves = 500, max_depth = 10, learning_rate = 0.09
        )
        model.fit(X_train, y_train)
        
        scores.append(corr(y_valid, model.predict(X_valid), np.array([asset.loc[asset_id,'Weight']] * len(y_valid))))
        
        X_trains.append(X_train)
        y_trains.append(y_train)
        X_valids.append(X_valid)
        y_valids.append(y_valid)
        models.append(model)
        importance = np.append(importance, model.feature_importances_)
    
    print('CORR:{}'.format(np.mean(scores)))
    
    df_feature = pd.DataFrame({
        'features' :X_valid.columns,
        'importance' : np.mean(importance.reshape(n_splits, -1), axis=0)})
    df_feature = df_feature.sort_values(by='importance',ascending = False)

    plt.figure(figsize=(14, 4))
    sns.barplot(x='features', y='importance', data=df_feature)
    plt.title('Feature Importance')
    plt.tight_layout()
    plt.show()
    
    print('-' * 90)
    X_train_dict[asset_id] = X_trains
    y_train_dict[asset_id] = y_trains
    X_valid_dict[asset_id] = X_valids
    y_valid_dict[asset_id] = y_valids
    model_dict[asset_id] = models
    


### テストデータの予測
Cryptoコンペでは、データと一緒に提供されているgresearch_crypto Python moduleを使用する必要があります。<br>
これは、将来のデータを使用して予測を行わないようにするためであるようです。<br>
[API Introduction](https://www.kaggle.com/sohier/detailed-api-introduction)に記載の通り、predictionを行います。<br>

In [ ]:
import gresearch_crypto

env = gresearch_crypto.make_env()
iter_test = env.iter_test()

In [ ]:
df_test_entire = []

# from the competition tutorial
for i, (df_test, sample_prediction_df) in enumerate(iter_test):
    for j, test_row in df_test.iterrows():
        try:
            cur_models = model_dict[test_row['Asset_ID']]
            X_test = cal_features(test_row).values.reshape(1, -1)
            y_pred = np.mean([model.predict(X_test) for model in cur_models])
        except: 
            y_pred = 0.0
        sample_prediction_df.loc[sample_prediction_df['row_id'] == test_row['row_id'], 'Target'] = y_pred
    df_test_entire.append(df_test)

    # Send submissions
    env.predict(sample_prediction_df)